In [62]:
STATIONSLINK= "https://en.wikipedia.org/wiki/List_of_rapid_transit_stations_in_Bangkok"

In [63]:
import requests
import re
from bs4 import BeautifulSoup

In [64]:
soup = BeautifulSoup(requests.get(STATIONSLINK).text, "lxml")

In [65]:
links = [row.select_one('td:nth-of-type(2) a') for row in soup.select('table.sortable tbody tr')]
links = set([a['href']for a in links if a])

links, len(links)

({'/wiki/11th_Infantry_Regiment_BTS_station',
  '/wiki/Ari_BTS_Station',
  '/wiki/Asok_BTS_Station',
  '/wiki/Ban_Thap_Chang_Station_(Airport_Rail_Link)',
  '/wiki/Bang_Bamru_Station',
  '/wiki/Bang_Bua_BTS_station',
  '/wiki/Bang_Chak_BTS_Station',
  '/wiki/Bang_Chan_MRT_station',
  '/wiki/Bang_Kapi_MRT_station',
  '/wiki/Bang_Khae_MRT_station',
  '/wiki/Bang_Khen_Station',
  '/wiki/Bang_Khun_Non_MRT_Station',
  '/wiki/Bang_Krasor_MRT_Station',
  '/wiki/Bang_Na_BTS_Station',
  '/wiki/Bang_O_MRT_Station',
  '/wiki/Bang_Phai_MRT_station',
  '/wiki/Bang_Phlat_MRT_Station',
  '/wiki/Bang_Phlu_MRT_Station',
  '/wiki/Bang_Pho_MRT_Station',
  '/wiki/Bang_Rak_Noi_Tha_It_MRT_Station',
  '/wiki/Bang_Rak_Yai_MRT_Station',
  '/wiki/Bang_Son_MRT_Station',
  '/wiki/Bang_Son_MRT_station',
  '/wiki/Bang_Sue_MRT_Station',
  '/wiki/Bang_Wa_BTS_Station',
  '/wiki/Bang_Wa_MRT_station',
  '/wiki/Bang_Yi_Khan_MRT_Station',
  '/wiki/Bearing_BTS_Station',
  '/wiki/Bhumibol_Adulyadej_Hospital_BTS_station',
  

In [66]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_station_info(wiki_url):
    """Fetch coordinates (English or Thai Wikipedia) & Thai station name."""
    headers = {"User-Agent": "Mozilla/5.0"}  # Prevents blocks
    response = requests.get(wiki_url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch {wiki_url}")
        return None, None, None

    soup = BeautifulSoup(response.text, "html.parser")

    # Extract all train lines
    line_elements = soup.select("td.category a[title]")  
    lines = [line["title"] for line in line_elements]  # Extract title attributes
    lines = ",".join(lines) if lines else None

    # Try extracting coordinates from English Wikipedia
    coordinates = soup.select_one("#coordinates span.geo-dec")
    coordinates = coordinates.get_text(strip=True) if coordinates else None

    # Find the Thai Wikipedia link
    thai_link = soup.select_one("li.interwiki-th a")

    if thai_link:
        thai_url = thai_link["href"]
        thai_name, thai_coordinates, thai_lines = get_thai_station_info(thai_url)

        lines = lines or thai_lines
        # If coordinates were missing in English, use Thai coordinates
        coordinates = coordinates or thai_coordinates  
        return thai_name, coordinates , lines   
    if coordinates:
        return None, coordinates, lines    # No Thai page, only coordinates found
    return None,None, lines  

def get_thai_station_info(thai_url):
    """Fetch station name and coordinates from Thai Wikipedia."""
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(thai_url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch Thai Wikipedia: {thai_url}")
        return "Unknown Station", None

    soup = BeautifulSoup(response.text, "html.parser")

    # Extract Thai station name
    thai_name = soup.select_one("#firstHeading span.mw-page-title-main")
    thai_name = thai_name.get_text(strip=True) if thai_name else "Unknown Station"

    # Extract coordinates from Thai Wikipedia
    thai_coordinates = soup.select_one("#coordinates span.geo-dec")
    thai_coordinates = thai_coordinates.get_text(strip=True) if thai_coordinates else None
    
    line_elements = soup.select("td.category a[title]")  
    lines = [line["title"] for line in line_elements]  # Extract title attributes
    lines = ",".join(lines) if lines else None

    return thai_name, thai_coordinates, lines 

# List of station Wikipedia links (Replace with your actual list)
station_links = [
    "https://en.wikipedia.org/wiki/Siam_BTS_station",
    "https://en.wikipedia.org/wiki/Mo_Chit_BTS_station",
    "https://en.wikipedia.org/wiki/Kheha_BTS_station",
    # Add more station links here...
]

data = []
for link in station_links:
    thai_name, coords, lines = get_station_info(link)
    data.append({"Station Name (Thai)": thai_name, "Coordinates": coords, "Line": lines})

# Convert to Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df



,Station Name (Thai),Coordinates,Line
0,สถานีสยาม,13.7456194°N 100.5342278°E,"รถไฟฟ้าบีทีเอส สายสุขุมวิท,รถไฟฟ้าบีทีเอส สายสีลม"
1,สถานีหมอชิต,13.8025833°N 100.5538333°E,Sukhumvit Line
2,สถานีเคหะฯ,13.5677°N 100.6077°E,Sukhumvit Line


In [67]:
# Scrape data for all stations
data = []
for link in links:
    thai_name, coords, lines = get_station_info("https://en.wikipedia.org"+link)
    data.append({"Station Name (Thai)": thai_name, "Coordinates": coords, "Lines": lines})

# Convert to Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df



,Station Name (Thai),Coordinates,Lines
0,สถานีบางซ่อน,13.8202°N 100.5324°E,"MRT (Bangkok),Purple Line (Bangkok)"
1,สถานีบางหว้า,13.7206250°N 100.4578694°E,"BTS Skytrain,Silom Line,MRT (Bangkok),Blue Lin..."
2,สถานีทองหล่อ,13.7242444°N 100.5785583°E,Sukhumvit Line
3,สถานีลาดพร้าว 101,13.7747°N 100.6298°E,Yellow Line (Bangkok)
4,สถานีกระทรวงสาธารณสุข,None,"MRT (Bangkok),Purple Line (Bangkok)"
...,...,...,...
183,สถานีรังสิต,13.9905°N 100.6022°E,รถไฟฟ้าชานเมือง สายธานีรัถยา
184,สถานีบางรักน้อยท่าอิฐ,13.8748°N 100.4559°E,"MRT (Bangkok),Purple Line (Bangkok)"
185,สถานีรัชดาภิเษก,13.7991°N 100.5750°E,"MRT (Bangkok),Blue Line (Bangkok)"
186,สถานีทิพวัล,13.6368°N 100.6099°E,Yellow Line (Bangkok)


In [68]:
duplicates = df[df["Station Name (Thai)"].duplicated(keep=False)]
print(duplicates)
df.drop_duplicates(subset=['Station Name (Thai)'], inplace=True)
df[df["Coordinates"].isna()]


          Station Name (Thai)                 Coordinates  \
0                สถานีบางซ่อน        13.8202°N 100.5324°E   
1                สถานีบางหว้า  13.7206250°N 100.4578694°E   
11               สถานีบางซ่อน        13.8202°N 100.5324°E   
18   สถานีกลางกรุงเทพอภิวัฒน์      13.80500°N 100.54167°E   
23                 สถานีพญาไท  13.7569417°N 100.5338444°E   
29               สถานีหัวหมาก                        None   
45      สถานีวัดพระศรีมหาธาตุ        13.8752°N 100.5967°E   
67                 สถานีสำโรง    13.647363°N 100.596153°E   
69               สถานีหัวหมาก                        None   
98                 สถานีพญาไท  13.7569417°N 100.5338444°E   
106              สถานีหลักสี่    13.884348°N 100.580793°E   
110                สถานีสำโรง    13.647363°N 100.596153°E   
143  สถานีกลางกรุงเทพอภิวัฒน์      13.80500°N 100.54167°E   
153     สถานีวัดพระศรีมหาธาตุ        13.8752°N 100.5967°E   
155              สถานีบางหว้า  13.7206250°N 100.4578694°E   
156   สถานีศูนย์ราชการนน

,Station Name (Thai),Coordinates,Lines
4,สถานีกระทรวงสาธารณสุข,None,"MRT (Bangkok),Purple Line (Bangkok)"
29,สถานีหัวหมาก,None,"Airport Rail Link (Bangkok),Airport Rail Link ..."
63,สถานีท่าพระ,None,"MRT (Bangkok),Blue Line (Bangkok)"
133,สถานีการเคหะ,None,รถไฟฟ้าชานเมือง สายธานีรัถยา


In [69]:
df.loc[df["Station Name (Thai)"] == 'สถานีการเคหะ', 'Coordinates'] = "13.898405207°N, 100.588840478°E"
df.loc[df["Station Name (Thai)"] == 'สถานีหัวหมาก', 'Coordinates']  = "13.7373952°N 100.64120893°E"
df.loc[df["Station Name (Thai)"] == 'สถานีท่าพระ', 'Coordinates']  = "13.7297884762°N, 100.474162996°E"
df.loc[df["Station Name (Thai)"] == 'สถานีกระทรวงสาธารณสุข', 'Coordinates']  = "13.84870092°N, 100.51469189°E"


In [70]:
df[df['Lines'].isna()]

,Station Name (Thai),Coordinates,Lines
18,สถานีกลางกรุงเทพอภิวัฒน์,13.80500°N 100.54167°E,None
149,สถานีสุรศักดิ์,13.7193°N 100.5214°E,None


In [71]:
df[df['Lines'].isna()]
df.loc[df["Station Name (Thai)"] == 'สถานีกลางกรุงเทพอภิวัฒน์', 'Lines']  = "รถไฟฟ้าชานเมือง สายธานีรัถยา,Blue Line"
df.loc[df["Station Name (Thai)"] == 'สถานีสุรศักดิ์', 'Lines']  = "Silom Line"

In [72]:
df['Lines'] = df["Lines"].str.replace("(Bangkok)", "").str.replace("BTS Skytrain,","")
all_Lines = df["Lines"].str.split(",").explode()  # Flatten the list
Lines_counts = all_Lines.value_counts()  # Count unique occurrences

# Display the result
print(Lines_counts)

Lines
MRT                              56
Sukhumvit Line                   46
Blue Line                        36
Pink Line                        29
Yellow Line                      22
Purple Line                      18
Airport Rail Link                14
รถไฟฟ้าชานเมือง สายธานีรัถยา     10
รถไฟฟ้าบีทีเอส สายสีลม            8
Silom Line                        6
Orange Line                       6
Gold Line                         2
รถไฟฟ้าชานเมือง สายนครวิถี        2
รถไฟฟ้ามหานคร สายเฉลิมรัชมงคล     1
Blue Line                         1
รถไฟฟ้าบีทีเอส สายสุขุมวิท        1
รถไฟฟ้ามหานคร สายสีชมพู           1
รถไฟฟ้ามหานคร สายนัคราพิพัฒน์     1
รถไฟฟ้าสายสีทอง                   1
รถไฟฟ้าแอร์พอร์ต เรล ลิงก์        1
Name: count, dtype: int64


In [73]:
lines_name = {'รถไฟฟ้าบีทีเอส สายสีลม': "Silom Line",
              'รถไฟฟ้าบีทีเอส สายสุขุมวิท': 'Sukhumvit Line',
              'รถไฟฟ้าแอร์พอร์ต เรล ลิงก์': 'Airport Rail Link',
              'รถไฟฟ้าสายสีทอง': 'Gold Line',
              'รถไฟฟ้ามหานคร สายเฉลิมรัชมงคล': 'Blue Line',
              'รถไฟฟ้ามหานคร สายนัคราพิพัฒน์': 'Yellow Line',
              'รถไฟฟ้ามหานคร สายสีชมพู': 'Pink Line',
              'รถไฟฟ้าชานเมือง สายนครวิถี' :'Red Line',
              'รถไฟฟ้าชานเมือง สายธานีรัถยา':'Red Line'
              }

def fit_lines(lines):
    lines = set(lines.split(','))
    lines_arr = []
    for line in lines:
        if 'MRT' not in line:
            if line in lines_name:
                lines_arr.append(lines_name[line])
            else:
                lines_arr.append(line.strip())
    return lines_arr


df["Lines"] = df["Lines"].apply(fit_lines)

In [74]:
all_Lines = df["Lines"].explode()  # Flatten the list
Lines_counts = all_Lines.value_counts()  # Count unique occurrences

# Display the result
print(Lines_counts), all_Lines.shape

Lines
Sukhumvit Line       47
Blue Line            38
Pink Line            30
Yellow Line          23
Purple Line          18
Silom Line           14
Red Line             12
Airport Rail Link     8
Orange Line           6
Gold Line             3
Name: count, dtype: int64


(None, (199,))

In [ ]:
# Save to CSV
df.to_csv("data/bangkok_bts_stations.csv", index=False)